In [63]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import bisect

torch.manual_seed(1337)

n_features = 1

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 33 # what is the maximum context length for predictions?
max_iters = 5000
epochs = 3
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
st_dims = 8
ac_dims = 1
rw_dims = 1
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab = 100

In [64]:
cols = ["Unnamed: 0", "X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "index"]
cols2 = ["Unnamed: 0", "observations", "next_observations", "index"]
obs_cols = ["X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "actions", "rewards", "rewardstg"]
renames = {'Action':'actions', 'Reward':'rewards', 'observation_x':'observations', 'observation_y':'next_observations'}

In [65]:

def get_data():
    dataframe = pd.read_csv("medium_play.csv")
    dataframe['observation'] = list(dataframe.iloc[:, 1:9].values)
    observations = dataframe['observation'].copy()
    observations.drop(observations.index[0], inplace=True)
    dataframe.drop(dataframe.index[-1], inplace=True)
    observations = observations.reset_index()
    merged_data = dataframe.merge(observations, left_index=True, right_index=True)
    merged_data["terminals"] = False
    merged_data.loc[184753:, "Episode"] += 2000
    grouped_episodes = merged_data.groupby("Episode")
    merged_data = merged_data.rename(columns=renames)
    for episode, group in grouped_episodes:
        last_index = group.index[-1]  # Get the index of the last row in the group
        merged_data.loc[last_index, 'terminals'] = True
        every = group.index
        merged_data.loc[every, 'rewardstg'] = merged_data.loc[every, 'rewards'].cumsum()
        merged_data.loc[every, 'rewardstg'] -= merged_data.loc[every[-1], 'rewardstg']
        merged_data.loc[every, 'rewardstg'] *= -1
    observations = np.vstack(merged_data['observations'])
    next_observations = np.vstack(merged_data['next_observations'])
    actions = torch.tensor(merged_data["actions"].values, dtype=torch.long)
    rewards = torch.tensor(merged_data["rewardstg"].values, dtype=torch.float)
    observations = torch.tensor(observations, dtype=torch.float)
    next_observations = torch.tensor(next_observations)
    merged_data.drop(cols2, inplace=True, axis=1)
    return merged_data, actions, rewards, observations


data, actions, rewards, observations = get_data()



In [66]:
#this class requres SAR sequence 
class UniformDiscretizer():
    def __init__(self, dataframe, vocab):
        self.columns = dataframe.columns
        self.mins = []
        self.maxs = []
        self.steps = []
        self.offsets = []
        self.data = dataframe
        self.vocab = vocab
        self.total_vocab = 0
        
    def fit_transform(self):
        discretized, targets = pd.DataFrame(), pd.DataFrame()
        t_vocab = 0
        
        for i, col in enumerate(self.columns):
            self.offsets.append(t_vocab)
            
            mini, maxi = self.data[col].min(), self.data[col].max()
            
            step = (maxi - mini)/(vocab-1)
            
            targets[col] = ((data[col] - mini) / step).astype(int)
            discretized[col] = self.offsets[-1] + targets[col]
            
            t_vocab += targets[col].nunique()
            
            self.mins.append(mini), self.maxs.append(maxi), self.steps.append(step)
            
        self.offsets.append(t_vocab)
        
        self.total_vocab = self.offsets[-1]
        
        return discretized, targets
    
    def trajectory_to_discretized(self, trajectory):
        return trajectory + offsets[:-1]
    
    def trajectory_to_target(self, trajectory):
        return trajectory - offsets[:-1]
    
    def token_to_discretized(self, token, column_idx):
        return token + self.offsets[column_idx]
        
    def token_to_target(self, token):
        column_idx = bisect.bisect_left(self.offsets, token) - 1
        return token - self.offsets[column_idx]
    
    def target_recover(self, token, column_idx):
        return (token+1)*self.steps[column_idx] + self.mins[column_idx]

In [67]:
train_df = data[obs_cols]
UD = UniformDiscretizer(train_df, 100)
discretized, targets = UD.fit_transform()
discretized = torch.tensor(discretized.values, device=device, dtype=torch.long).reshape(-1)
targets = torch.tensor(targets.values, device=device, dtype=torch.long).reshape(-1)
total_vocab = UD.total_vocab
trajectory_length = len(UD.columns)

In [74]:
def get_batch():
    ix = torch.randint(len(discretized) - block_size, (batch_size,))
    x = torch.stack([discretized[i:i+block_size] for i in ix])
    y = torch.stack([targets[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    tt.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        x, y = get_batch()
        logits, loss = tt(x, targets=y)
        losses[k] = loss.item()
    out = losses.mean()
    tt.train()
    return out

In [98]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(3*block_size, 3*block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [103]:
# super simple bigram model
class TrajectoryTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.init_embd = nn.Embedding(total_vocab, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab)

    def forward(self, sequence, targets=None):
        vocab_embedding = self.init_embd(sequence)#(B,T,C)
        B, T, C = vocab_embedding.shape
        pos_encoding = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = vocab_embedding + pos_encoding #(B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)

        preds = self.lm_head(x) # (B,T,vocab)

        if targets is None:
            loss = None
        else:
            B, T, C = preds.shape
            preds = preds.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(preds, targets)

        return preds, loss

In [104]:
def beam_search(model, context, st_width=3, ac_width=4, horizon=1, tr_width=1):
    window_by_index = [st_width]*st_dims + [ac_width]*ac_dims + [1] + [1]
    top_beams = [[context, 0]]
    square_beams = []
    for i in range(trajectory_length):
        k = window_by_index[i]
        for beam in top_pairs:
            seq, prob = beam
            logits, __ = model(seq[-block_size:])
            logits = logits[-1, :]
            top_k = np.argpartition(logits, -k)[-k:]
            temp_beams = [[seq + [UD.token_to_discretize(tk_idx, i)], prob + logits[tk_idx]] for tk_idx in top_k]
            square_beams += temp_beams
            
        top_beams = sorted(square_beams, key=lambda x: x[1])
        if i < st_dims:
            top_beams = top_beams[-k:]
        #probably incorporate i < st_dims + ac_dims toop_beams -st_width * ac_width
      
    top_rwds_idx = np.argsort([target_recover(x[0][-2], -2) + target_recover([0][-1], -1) for x in top_beams])
    return top_beams[top_rwds_idx[-tr_width:]]

def extract_action(trajectory):
    return trajectory[st_dims]

In [105]:
tt = TrajectoryTransformer()
optimizer = torch.optim.AdamW(tt.parameters(), lr=learning_rate)

In [106]:
for _ in range(epochs):
    for iteration in range(max_iters):
      # every once in a while evaluate the loss on train and val sets
        if iteration % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iteration}: train loss {losses:.4f}")

        # sample a batch of data
        x, y = get_batch()

        # evaluate the loss
        logits, loss = tt(x, targets=y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

step 0: train loss 4.7752
step 100: train loss 2.2729
step 200: train loss 2.0934
step 300: train loss 1.9541
step 400: train loss 1.8933


KeyboardInterrupt: 

# Testing

In [ ]:
import gymnasium as gym

In [ ]:
def testing(model, desired_reward, iterations):
    env = gym.make("LunarLander-v2")
    copy_dr = desired_reward
    for i in range(iterations):
        # Reset it
        total_reward = 0.0
        desired_reward = copy_dr
        #numpy array (C)
        state, _ = env.reset(seed=None)

        # While the episode is not finished
        terminated = False
        time = 0

        rewards = torch.full((1, rw_dims), desired_reward, dtype=torch.float, device=device)
        states = torch.zeros((1, 1, st_dims), dtype=torch.float, device=device)
        states[0, 0, :] = torch.tensor(state, dtype=torch.float, device=device)
        actions = torch.zeros((1, 1), dtype=torch.long, device=device)
        actions[0, 0] = 0.0

        # define a trigger function, return True to start recording a new video:
        episode_trigger=lambda episode_id: (episode_id%1==0)

        while not terminated and time <= 500:
            input_st, input_ac, input_rw = states[:, -block_size:, :], \
                  actions[:, -block_size:], rewards[:, -block_size:]

            output, loss = dt(input_st, input_ac, input_rw)
            output = output[:, -1:, :].view(ac_dims)
            probabilities = F.softmax(output[:], dim=0)
            # ---> TODO: how to select an action
            # select action with the greatest probability according to policy
            action = int(torch.argmax(probabilities))
            # One step forward
            state, reward, terminated, _, _ = env.step(action)

            total_reward += reward
            desired_reward -= reward
            state = torch.tensor(state, dtype=torch.float, device=device).view(1, 1, st_dims)
            states = torch.cat((states, state), dim=1)

            reward = torch.tensor(desired_reward, dtype=torch.float, device=device).view(1, rw_dims)
            rewards = torch.cat((rewards, reward), dim=1)

            action = torch.tensor(action, dtype=torch.long, device=device).view(1, 1)
            actions = torch.cat((actions, action), dim=1)

            time += 1

      # Print reward
        print("total_reward = {}".format(total_reward))
    env.close()

In [ ]:
testing(dt, 200, 10)

In [ ]:
torch.save(dt, "first_iter.model")

# Load Model